In [ ]:
0

In [ ]:
cd ..

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import subprocess
import footsteps
import os
import glob
from PIL import Image
import numpy as np
import tqdm
import torch

def DavisEval(model, name, partial=False):
    davis_path = "data_storage/DAVIS/"
    output_path = footsteps.output_dir + name + "/"
    
    os.mkdir(output_path)
    with open(davis_path + "/ImageSets/2017/val.txt", "r") as f:
        sequences = f.readlines()
    if partial:
        sequences = sequences[8:30]
    for sequence in tqdm.tqdm(sequences):
        sequence = sequence[:-1]  # strip newline
        sequence_out_path = output_path + sequence + "/"
        sequence_img_path = davis_path + "JPEGImages/480p/" + sequence + "/"
        with open(davis_path + "Annotations/480p/" + sequence + "/00000.png", "rb") as handle:
          first_annotation = np.array(
            Image.open(handle)
          )

        os.mkdir(output_path + sequence)
        with open(sequence_img_path + "00000.jpg", "rb") as handle:
          first_image = np.array(Image.open(handle))

        prev_image = first_image
        prev_annotation = first_annotation
        
        if partial:
            frames = range(1, len(os.listdir(sequence_img_path)))[::5]
        else:
            frames = range(1, len(os.listdir(sequence_img_path)))

        for i in frames:
            
            with open(sequence_img_path + f"{i:05}.jpg", "rb") as handle:
              curr_image = np.array(Image.open(handle))
            annotation = model(
                first_image, first_annotation, prev_image, prev_annotation, curr_image
            )
            Image.fromarray(annotation).save(sequence_out_path + f"{i:05}.png")

            prev_image = curr_image
            prev_annoration = first_annotation
    
    subprocess.run(
        [
            "python",
            "davis2017-evaluation/evaluation_method.py",
            "--davis_path",
            "data_storage/DAVIS",
            "--results_path",
            output_path,
            "--task",
            "semi-supervised",
        ]
    )


In [ ]:
import fmapicon.models

In [ ]:

import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
l = []

def execute_model(A, B, model):
    N = A.shape[0]
    SIDE_LENGTH = A.shape[2]
    
    
    A = torch.tensor(A).cuda().permute(0, 3, 1, 2).float()[:, [2, 1, 0]]
    B = torch.tensor(B).cuda().permute(0, 3, 1, 2).float()[:, [2, 1, 0]]
    feats_a = model(A)
    feats_b = model(B)

    # to (N, C, H*W)
    feats_a = feats_a.reshape(feats_a.shape[0],
        feats_a.shape[1], feats_a.shape[2] * feats_a.shape[3])
    feats_b = feats_b.reshape(feats_b.shape[0],
        feats_b.shape[1], feats_b.shape[2] * feats_b.shape[3])

    feats_a = feats_a.permute(0, 2, 1)

    #return feats_a
    cc = torch.bmm(feats_a, feats_b)

    cc = nn.functional.softmax(cc, dim=-1)


    cc = cc.reshape([N] + [SIDE_LENGTH] * 4)
    #cc = np.array(cc.cpu().detach())
    
    cc2 = cc.reshape([N, SIDE_LENGTH, SIDE_LENGTH, SIDE_LENGTH**2])
    
    index_grid = torch.argmax(cc2, axis=-1).cpu().detach()[:, :, :, None].numpy()
    
    


    grid = np.concatenate([index_grid % SIDE_LENGTH, index_grid / SIDE_LENGTH], axis=-1)
    
    return cc, grid

class FMAPICON_model:
    def __init__(self, weights_path, do_vis = False):
        self.inner_model = fmapicon.models.tallUNet64().cuda()
        self.inner_model.eval()
        self.inner_model.load_state_dict(torch.load(weights_path))
        self.do_vis = do_vis
        
    def __call__(self, initial_frame, initial_mask, prev_frame, prev_mask, current_frame):
        crop = (initial_frame.shape[1] - initial_frame.shape[0]) // 2
        B = initial_frame[None, ::4, crop:-crop:4] / 255.   
        A = current_frame[None, ::4, crop:-crop:4] / 255.
        
        cc, grid = execute_model(A, B, self.inner_model)
        m = initial_mask[2::4, 2+crop:-crop:4]

        shitty_res = m[grid[0, :, :, 1].astype(int), grid[0, :, :, 0].astype(int)]

        up_res = np.repeat(np.repeat(shitty_res, 4, axis=0),4, axis=1)
        
        out_mask = initial_mask.copy()
        
        out_mask[:, crop:-crop] = up_res
        if self.do_vis:
            plt.imshow(A[0])
            plt.show()
            plt.imshow(B[0])
            plt.show()
            plt.imshow(shitty_res)
            plt.show()
            plt.imshow(out_mask)
            plt.show()
            #l.append(up_res)

            plt.imshow(grid[0, :, :, 0])
            plt.show()

            #l.append((cc.detach().cpu(), A, B, initial_mask))
        
        

        return out_mask

#DavisEval(FMAPICON_model("tensorflow_/results/log_probability_2/epoch50/model_weights.tf"), "FMAPICON")
#DavisEval(FMAPICON_model("tensorflow_/results/clip.6_11/epoch51/model_weights.tf"), "FMAPICON")

In [ ]:
vis = False

DavisEval(FMAPICON_model(f"results/rolling_augmentation_2/network00050.trch", vis), f"rolling_test", vis)

In [ ]:
vis = False

DavisEval(FMAPICON_model(f"results/batch_norm_evverrwer/network00080.trch", vis), f"bnormv", vis)

 93%|█████████▎| 28/30 [01:28<00:06,  3.39s/it]

In [ ]:
vis = True
DavisEval(FMAPICON_model(f"results/clip_gradient_mannequin/network00043.trch", vis), f"manneq_bgr4", vis)

In [ ]:
vis = True
DavisEval(FMAPICON_model(f"results/pixelwise_norm_7/network00086.trch", vis), f"dod738od990o", vis)

In [ ]:
DavisEval(FMAPICON_model(f"results/kinetics_train/network00005.trch"), f"kinetics")

In [ ]:
model = FMAPICON_model(f"results/clip_norm/network00041.trch").inner_model

In [ ]:
cc,A,B,m=l[0]

In [ ]:
cc, grid = execute_model(A, B, model)

In [ ]:
import fmapicon.training
feats_A = fmapicon.training.warping(model, torch.tensor(A).cuda().permute(0, 3, 1, 2).float()).detach().cpu().numpy()
feats_B = model(torch.tensor(B).cuda().permute(0, 3, 1, 2).float()).detach().cpu().numpy()

In [ ]:
plt.figure()
plt.imshow(feats_A[0, 7])
plt.title("Edge artifacts from warping and unwarping.\n Must be left in or training fails")
plt.colorbar()
plt.figure()
plt.title("cropped feature map\n from network trained with cropping")
plt.imshow(feats_A[0, 3, 20:-20, 20:-20])
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.sum(feats_A[0]**2, axis=0))
plt.colorbar()

In [ ]:
plt.imshow(feats_A[0 , 18])
plt.colorbar()
plt.show()
plt.imshow(feats_B[0 , 18])
plt.colorbar()

In [ ]:
#plt.imshow(grid[0, :, :, 0].transpose())
#plt.show()
plt.imshow(A[0])
plt.show()
plt.imshow(B[0])
plt.show()

In [ ]:
0